# Microsoft Malware Prediction 

## MLflow

### Importamos las librerías

In [3]:
%sh
pip install mlflow

Requirement already satisfied: mlflow in /databricks/python3/lib/python3.7/site-packages (1.8.0)
Requirement already satisfied: pandas in /databricks/python3/lib/python3.7/site-packages (from mlflow) (0.24.2)
Requirement already satisfied: querystring-parser in /databricks/python3/lib/python3.7/site-packages (from mlflow) (1.2.4)
Requirement already satisfied: sqlalchemy<=1.3.13 in /databricks/python3/lib/python3.7/site-packages (from mlflow) (1.3.13)
Requirement already satisfied: protobuf>=3.6.0 in /databricks/python3/lib/python3.7/site-packages (from mlflow) (3.12.1)
Requirement already satisfied: simplejson in /databricks/python3/lib/python3.7/site-packages (from mlflow) (3.17.0)
Requirement already satisfied: python-dateutil in /databricks/python3/lib/python3.7/site-packages (from mlflow) (2.8.0)
Requirement already satisfied: gitpython>=2.1.0 in /databricks/python3/lib/python3.7/site-packages (from mlflow) (3.1.2)
Requirement already satisfied: Flask in /databricks/python3/lib/python3.7/site-packages (from mlflow) (1.1.2)
Requirement already satisfied: docker>=4.0.0 in /databricks/python3/lib/python3.7/site-packages (from mlflow) (4.2.0)
Requirement already satisfied: gorilla in /databricks/python3/lib/python3.7/site-packages (from mlflow) (0.3.0)
Requirement already satisfied: six>=1.10.0 in /databricks/python3/lib/python3.7/site-packages (from mlflow) (1.12.0)
Requirement already satisfied: prometheus-flask-exporter in /databricks/python3/lib/python3.7/site-packages (from mlflow) (0.13.0)
Requirement already satisfied: pyyaml in /databricks/python3/lib/python3.7/site-packages (from mlflow) (5.3.1)
Requirement already satisfied: click>=7.0 in /databricks/python3/lib/python3.7/site-packages (from mlflow) (7.1.2)
Requirement already satisfied: databricks-cli>=0.8.7 in /databricks/python3/lib/python3.7/site-packages (from mlflow) (0.10.0)
Requirement already satisfied: sqlparse in /databricks/python3/lib/python3.7/site-packages (from mlflow) (0.3.1)
Requirement already satisfied: gunicorn; platform_system != "Windows" in /databricks/python3/lib/python3.7/site-packages (from mlflow) (20.0.4)
Requirement already satisfied: numpy in /databricks/python3/lib/python3.7/site-packages (from mlflow) (1.16.2)
Requirement already satisfied: alembic in /databricks/python3/lib/python3.7/site-packages (from mlflow) (1.4.2)
Requirement already satisfied: entrypoints in /databricks/python3/lib/python3.7/site-packages (from mlflow) (0.3)
Requirement already satisfied: cloudpickle in /databricks/python3/lib/python3.7/site-packages (from mlflow) (1.4.1)
Requirement already satisfied: requests>=2.17.3 in /databricks/python3/lib/python3.7/site-packages (from mlflow) (2.21.0)
Requirement already satisfied: pytz>=2011k in /databricks/python3/lib/python3.7/site-packages (from pandas->mlflow) (2018.9)
Requirement already satisfied: setuptools in /usr/lib/python3.7/site-packages (from protobuf>=3.6.0->mlflow) (40.8.0)
Requirement already satisfied: gitdb<5,>=4.0.1 in /databricks/python3/lib/python3.7/site-packages (from gitpython>=2.1.0->mlflow) (4.0.5)
Requirement already satisfied: Jinja2>=2.10.1 in /databricks/python3/lib/python3.7/site-packages (from Flask->mlflow) (2.11.2)
Requirement already satisfied: Werkzeug>=0.15 in /databricks/python3/lib/python3.7/site-packages (from Flask->mlflow) (1.0.1)
Requirement already satisfied: itsdangerous>=0.24 in /databricks/python3/lib/python3.7/site-packages (from Flask->mlflow) (1.1.0)
Requirement already satisfied: websocket-client>=0.32.0 in /databricks/python3/lib/python3.7/site-packages (from docker>=4.0.0->mlflow) (0.57.0)
Requirement already satisfied: prometheus-client in /databricks/python3/lib/python3.7/site-packages (from prometheus-flask-exporter->mlflow) (0.7.1)
Requirement already satisfied: configparser>=0.3.5 in /databricks/python3/lib/python3.7/site-packages (from databricks-cli>=0.8.7->mlflow) (5.0.0)
Requirement already satisfied: tabulate>=0.7.7 in /databricks/python3/lib/python3.7/site

In [4]:
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
import pickle
import azureml
import mlflow.azureml

from azureml.core import Workspace
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn import metrics
from azure.storage.blob import BlockBlobService
from azureml.core.webservice import AciWebservice, Webservice

### Descargar los datos

In [6]:
# Descargamos todos los ficheros del contenedor de datos

# 1. Nos autenticamos
STORAGEACCOUNTNAME = "datostfm"
STORAGEACCOUNTKEY = "BAypoQhwJHFW/dwMM72rikWyxFlhFKmAlds6nQ7FXvuu0f1G1qBKWVW5wwrqI0lrq38hb35BAILSCtg/sgLObQ=="
CONTAINERNAME = "datos"
blob_service = BlockBlobService(account_name=STORAGEACCOUNTNAME, account_key=STORAGEACCOUNTKEY)

# 2. Descargamos del Blob
LOCALFILENAME = ["X_train.csv", "X_val.csv", "y_train.csv", "y_val.csv"]
BLOBNAME = LOCALFILENAME
for csv in LOCALFILENAME:
    blob_service.get_blob_to_path(CONTAINERNAME, csv, csv)

In [7]:
# Convertimos a pandas
X_train = pd.read_csv(LOCALFILENAME[0])
X_val = pd.read_csv(LOCALFILENAME[1])
y_train = pd.read_csv(LOCALFILENAME[2])
y_val = pd.read_csv(LOCALFILENAME[3])

### Descargar los modelos

In [9]:
# Descargamos los modelos del contenedor

# 1. Nos autenticamos
STORAGEACCOUNTNAME = "datostfm"
STORAGEACCOUNTKEY = "BAypoQhwJHFW/dwMM72rikWyxFlhFKmAlds6nQ7FXvuu0f1G1qBKWVW5wwrqI0lrq38hb35BAILSCtg/sgLObQ=="
CONTAINERNAME = "modelos"
blob_service = BlockBlobService(account_name=STORAGEACCOUNTNAME, account_key=STORAGEACCOUNTKEY)

# 2. Descargamos del Blob
LOCALFILENAME = ["random_forest.pkl", "regresion_logistica.pkl", "gradient_boosting.pkl"]
BLOBNAME = LOCALFILENAME
for csv in LOCALFILENAME:
    blob_service.get_blob_to_path(CONTAINERNAME, csv, csv)

In [10]:
# Cargamos el modelo para Random Forest
pkl_filenameRF = "random_forest.pkl"
with open(pkl_filenameRF, 'rb') as file:
    rf_model = pickle.load(file)

# Cargamos el modelo para Regresión Logística
pkl_filenameRL = "regresion_logistica.pkl"
with open(pkl_filenameRL, 'rb') as file:
    rl_model = pickle.load(file)
    
# Cargamos el modelo para Gradient Boosting
pkl_filenameGB = "gradient_boosting.pkl"
with open(pkl_filenameGB, 'rb') as file:
    gb_model = pickle.load(file)    

### Poner en producción los modelos

**Definir las métricas**

In [13]:
def eval_metricas(y_val, y_pred):
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    mae = mean_absolute_error(y_val, y_pred)
    logloss = metrics.log_loss(y_val, y_pred)
    accuracy = metrics.accuracy_score(y_val, y_pred)
    F1 = metrics.f1_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred, average='binary')
    recall = recall_score(y_val, y_pred, average='binary')
    auc = metrics.roc_auc_score(y_val, y_pred)
    
    return (rmse, mae, logloss, accuracy, F1, precision, recall, auc)

#### Algoritmo de Random Forest

Necesitamos primero **crear la instancia del modelo en MLflow**. Podemos ver cada una de las ejecuciones dentro de la GUI de Databricks, gracias a la integración de MLflow. Al hacer clic en cada entrada, se devuelven los detalles de la ejecución, los archivos que definen el modelo y mucho más. Y al hacer clic en la pestaña _Runs_ en la esquina superior derecha, observamos que podemos ver detalles sobre cada una de nuestras ejecuciones realizadas con MLflow.

In [16]:
# Iniciamos 'mlflow'
with mlflow.start_run():
    
    # Obtenemos las predicciones con el modelo 01  
    predict = rf_model.predict(X_val)
    
    # Calculamos diversas medidas
    (rmse, mae, logloss, accuracy, F1, precision, recall, auc) = eval_metricas(y_val, predict)

    # Log mlflow parámetros
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("logloss", logloss)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("F1", F1)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("auc", auc)

    # Log modelo generado
    mlflow.sklearn.log_model(rf_model, "RFmodel")

Ahora **cargamos nuestro modelo MLflow** (que se guardó en nuestro pipeline MLflow anteriormente) y lo cargamos en una variable para la puntuación por lotes (batch). Para encontrar la ruta del modelo que queremos desplegar, debemos ir a la información de la ejecución del modelo en MLflow.

`dbfs:/databricks/mlflow/443106500331548/654e005f60fd4040bbd62a0e9fc0e50f/artifacts/RFmodel`

In [18]:
model_path = "dbfs:/databricks/mlflow/443106500331548/654e005f60fd4040bbd62a0e9fc0e50f/artifacts/RFmodel"
dbutils.fs.ls(model_path)

Out[11]: [FileInfo(path='dbfs:/databricks/mlflow/443106500331548/654e005f60fd4040bbd62a0e9fc0e50f/artifacts/RFmodel/MLmodel', name='MLmodel', size=345),
 FileInfo(path='dbfs:/databricks/mlflow/443106500331548/654e005f60fd4040bbd62a0e9fc0e50f/artifacts/RFmodel/conda.yaml', name='conda.yaml', size=136),
 FileInfo(path='dbfs:/databricks/mlflow/443106500331548/654e005f60fd4040bbd62a0e9fc0e50f/artifacts/RFmodel/model.pkl', name='model.pkl', size=131472984)]

Antes de que los modelos se puedan implementar en Azure ML, se debe **crear un Azure ML Workspace**. La función `azureml.core.Workspace.create()` cargará un espacio de trabajo con un nombre especificado o creará uno si aún no existe (hacemos uso de un recurso ya creado, y buscamos los valores de la configuración que necesitamos). Cuando ejecutamos esa función, es necesario autenticarse, a través de la URL, y meter un código de autenticación.

In [20]:
workspace_name = "ML-TFM"
workspace_location = "southcentralus"
resource_group = "TFM_GroupResource"
subscription_id = "678e98bc-31b5-4562-9651-04fef641d4b3"

workspace = Workspace.create(name = workspace_name, subscription_id = subscription_id, resource_group = resource_group, 
                             location = workspace_location, exist_ok=True)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FKDGDLN7T to authenticate.
Interactive authentication successfully completed.

**Construimos nuestro modelo de Random Forest** previamente entrenado y almacenado en MLflow en una imagen desplegable de Docker usando solo 3 líneas de código. Para crear el contenedor, se hace uso de YAML a través del fichero que se nos crea con el modelo en MLflow. Con la ejecución, vemos que nuestro modelo se registra en Azure ML Workspace.

In [22]:
model_image, azure_model = mlflow.azureml.build_image(model_uri=model_path, 
                                                      workspace=workspace, 
                                                      model_name="model-rf-malware",
                                                      image_name="model-rf-malware-container-image",
                                                      description="mlflow rf model for scoring machine malware infection status",
                                                      synchronous=False)
model_image.wait_for_creation(show_output=True)

Registering model model-rf-malware
/databricks/python/lib/python3.7/site-packages/mlflow/azureml/__init__.py:202: DeprecationWarning: ContainerImage class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
 tags=tags,
/databricks/python/lib/python3.7/site-packages/azureml/core/image/container.py:159: DeprecationWarning: ContainerImageConfig class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
 base_image, base_image_registry, cuda_version=cuda_version)
/databricks/python/lib/python3.7/site-packages/mlflow/azureml/__init__.py:207: DeprecationWarning: Image class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
 models=[registered_model])
Creating image
/databricks/python/lib/python3.7/site-packages/azureml/core/image/image.py:401: DeprecationWarning: Image class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
 image = Image(workspace, id=image_id)
Running...................................................................
Succeeded
Image creation operation finished for image model-rf-malware-container-image:2, operation "Succeeded"

Una vez tenemos nuestro modelo, tenemos que pasar a **crear la implementación del modelo**. Para ello hay que crear un ACI (Azure Container Instances) y hacer el despliegue de servicio web utilizando la imagen del contenedor del modelo.

In [24]:
aci_webservice_name = "model-rf-malware-aci"
aci_webservice_deployment_config = AciWebservice.deploy_configuration()
aci_webservice = Webservice.deploy_from_image(name = aci_webservice_name, image = model_image, 
                                              deployment_config = aci_webservice_deployment_config, workspace = workspace)
aci_webservice.wait_for_deployment(show_output=True)

/local_disk0/tmp/1590233531351-0/PythonShell.py:4: DeprecationWarning: deploy_from_image has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
 import errno
/databricks/python/lib/python3.7/site-packages/azureml/core/image/image.py:815: DeprecationWarning: Image class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
 image = cls(None)
Running.................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"

Para desplegar los otros modelos, simplemente hay que realizar el mismo proceso que acabamos de comentar modificando el modelo que se quiera usar.